In [ ]:
import jax.numpy as jnp
import jax
import numpy as np
import time
import gymnasium as gym
import exciting_environments as excenv
#from exciting_environments import GymWrapper, EnvironmentRegistry
#import jax_dataclasses as jdc
from dataclasses import fields
from exciting_environments.utils import MinMaxNormalization
jax.config.update("jax_enable_x64", True)
import diffrax

In [ ]:
# pmsm_env=EnvironmentRegistry.PMSM.make(saturated=True,motor_variant=excenv.pmsm.MotorVariant.BRUSA,batch_size=2)
# pmsm_env2=EnvironmentRegistry("PMSM-v0").make(saturated=True,motor_variant=excenv.pmsm.MotorVariant("BRUSA"),batch_size=2)
#env=excenv.make('MassSpringDamper-v0',batch_size=2)

In [ ]:
pend_env=excenv.PMSM(saturated=False,LUT_motor_name="BRUSA") #saturated=False,LUT_motor_name="BRUSA"

## Step and Simulate ahead

In [ ]:
key=jax.random.PRNGKey(1234)
obs, state = pend_env.reset()

In [ ]:
obs,states,last_state=pend_env.sim_ahead(state,jnp.ones((4,2)))
obs

In [ ]:
pend_env.generate_rew_trunc_term_ahead(states,jnp.ones((4,2)))

In [ ]:
key=jax.random.PRNGKey(1234)
obs, state = pend_env.reset()
generated_observations = []
generated_actions= []
generated_observations.append(obs)
for i in range(4):
    key,subkey= jax.random.split(key)
    action = jnp.ones(2)#jax.random.uniform(subkey,(2,),minval=-1,maxval=1)
    obs, state = pend_env.step(state, action)
    generated_actions.append(action)
    generated_observations.append(obs)

In [ ]:
generated_observations

### Vmapped

In [ ]:
batch_size= 2
keys = jax.random.split(jax.random.PRNGKey(0), batch_size)
#envs = [Pendulum(solver=diffrax.Euler(),batch_size=1,static_params={"g": jnp.array(9.81), "l": jnp.array(float(1+i)),  "m": jnp.array(1.0)}) for i in range(4)]
#envs = [excenv.PMSM() for _ in range(batch_size)]
envs = [excenv.PMSM(saturated=False,LUT_motor_name="BRUSA"),excenv.PMSM(saturated=False,LUT_motor_name="BRUSA")]
batched_envs = jax.tree.map(lambda *args: jnp.stack(args), *envs)
batched_envs = excenv.make("PMSM-v0",batch_size=2)
single_env = excenv.make("PMSM-v0")

In [ ]:
batched_envs

In [ ]:
jnp.repeat(jnp.array(3),batch_size)

In [ ]:
keys = jax.random.split(jax.random.PRNGKey(0), batch_size)
# obs, states = jax.vmap(lambda e,k: e.reset(k))(batched_envs,keys)
# print(obs)
obs, states = batched_envs.vmap_reset(keys)
print(obs)

In [ ]:
# state_in=jax.vmap(lambda e,o,k: e.generate_state_from_observation(o,k))(batched_envs,obs,keys)
# print(state_in.physical_state.omega)
state_in=batched_envs.vmap_generate_state_from_observation(obs,keys)
print(state_in.physical_state.i_d)

In [ ]:
actions = jnp.ones((batch_size,5))
obs, states = batched_envs.vmap_reset()
# next_obs, next_states = jax.vmap(
#     lambda e, s, a: e.step(s, a)
# )(batched_envs, states, actions)
# print(next_obs)
next_obs, next_states = batched_envs.vmap_step(states, actions)
print(next_obs)

In [ ]:
keys = jax.random.split(jax.random.PRNGKey(0), batch_size)
obs, states = batched_envs.vmap_reset()
actions = jnp.ones((batch_size,4,2))
#obs,states,last_state=pend_env.sim_ahead(state,actions=jnp.ones((4,2)))
# next_obs, next_states, last_state = jax.vmap(
#     lambda e, s, a: e.sim_ahead(s, a)
# )(batched_envs, states, actions)
# print(next_obs)
next_obs, next_states, last_state = batched_envs.vmap_sim_ahead(states, actions)
print(next_obs)

In [ ]:
#print(jax.vmap(lambda e, s, a: e.generate_rew_trunc_term_ahead(s, a))(batched_envs, next_states, actions))
batched_envs.vmap_generate_rew_trunc_term_ahead(next_states, actions)